In [33]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np 
import pickle

In [34]:
next_words = 91
max_sequence_len = 15
total_words = 7228

In [35]:
token_path = "./sonnet_model/data/tokenizer.pkl"
with open(token_path, "rb") as fp:
  tokenizer = pickle.load(fp)

In [36]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)

In [37]:
model_path = "./sonnet_model/sonnet-model.hdf5"
model.load_weights(model_path)

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 14, 100)           722800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               301200    
_________________________________________________________________
dense_1 (Dense)              (None, 7228)              2175628   
Total params: 3,199,628
Trainable params: 3,199,628
Non-trainable params: 0
_________________________________________________________________


In [62]:
def genSonnet(seed_text):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word

    seed_text = seed_text.split(" ")
    
    i=1
    output = []
    temp = ""
    for word in seed_text: 
        if i%7 == 0:
            temp += "<eos>"
            output.append(temp)
            temp = ""
        temp += word + " "
        i = i+1
    return output

In [40]:
# estimating 7 words per line => 7*13 = 91
# seed_text = "The sky is bright and blue"

# genSonnet(seed_text)

In [53]:
output = ""
data = open("dataset_test.txt","r")

testdata = data.read().replace('\n', '')
data.close()

corpus = testdata.lower().split("<eos>")



In [54]:
len(corpus)

743

In [55]:
cnt =0
seed_text =[]
for line in corpus:
    if cnt%14 == 0:
        seed_text.append(line)
    cnt+=1

In [56]:
len(seed_text)

54

In [64]:
# cnt =0
# for seed in seed_text:
#     print(seed)
#     sonnet = genSonnet(seed)
#     print(sonnet)
#     cnt = cnt+1

In [65]:
len(seed_text)

54

In [66]:

for seed in seed_text:
    sonnet = genSonnet(seed)
    with open("Gen_test.txt", "a+") as file_object:
        appendEOL = False
        # Move read cursor to the start of file.
        file_object.seek(0)
        # Check if file is not empty
        data = file_object.read(100)
        if len(data) > 0:
            appendEOL = True
        # Iterate over each string in the list
        for line in sonnet:
            # If file is not empty then append '\n' before first line for
            # other lines always append '\n' before appending line
            if appendEOL == True:
                file_object.write("\n")
            else:
                appendEOL = True
            # Append element at the end of file
            file_object.write(line)
    

    

In [70]:
data = open("dataset_test.txt","r")
testdata = data.read()
len(testdata)

36706

In [72]:
testdata = testdata[:31029]

In [73]:
with open("dataset_test.txt", "w") as file_object:
    file_object.write(testdata)

In [74]:
data = open("Gen_test.txt","r")
testdata = data.read()

len(testdata)

31029

In [75]:
data = open("dataset_test.txt","r")
testdata = data.read()
len(testdata)

31029

In [76]:
from nlgeval import compute_metrics
metrics_dict = compute_metrics(hypothesis='dataset_test.txt',references=['Gen_test.txt'])

FileNotFoundError: [Errno 2] No such file or directory: 'java': 'java'